
# Lab — EDA Univariate Analysis: Amazon UK Product Insights

Run the cells top-to-bottom. This notebook explores categories, prices, and ratings from the Amazon UK dataset and creates a short business summary.



## 0) (Optional) Install libraries
Run only if you don't already have these packages.


In [ ]:

# If needed, uncomment and run:
# !pip install -q pandas numpy matplotlib



## 1) Load the dataset
- Download the dataset from the Kaggle link in your README.
- Put the CSV next to this notebook (or use a full path).
- Set `DATA_PATH` accordingly.


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np

# >>> CHANGE THIS to your actual CSV filename or full path
DATA_PATH = Path("amazon_uk_products.csv")

try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
except Exception as e:
    print("Default CSV load failed, trying engine='python'...\n", e)
    df = pd.read_csv(DATA_PATH, low_memory=False, engine='python')

print("Rows:", len(df), " Columns:", len(df.columns))
display(df.head(3))
print("\nColumns:", list(df.columns))



## 2) Map dataset columns
Auto-detect likely columns for category, price, and rating. Override if needed.


In [ ]:

def _normalize(name: str) -> str:
    return name.strip().lower().replace(" ", "_").replace("-", "_")

def guess_column(df, candidates):
    cols = { _normalize(c): c for c in df.columns }
    for cand in candidates:
        key = _normalize(cand)
        if key in cols:
            return cols[key]
        for k, orig in cols.items():
            if key in k:
                return orig
    return None

category_candidates = ["category", "categories", "product_category", "main_category", "item_category"]
price_candidates    = ["price", "current_price", "sale_price", "offer_price", "discounted_price", "listing_price", "amount"]
rating_candidates   = ["rating", "ratings", "stars", "review_rating", "avg_rating", "star_rating"]

col_category = guess_column(df, category_candidates)
col_price    = guess_column(df, price_candidates)
col_rating   = guess_column(df, rating_candidates)

print("Auto-detected:")
print("  category ->", col_category)
print("  price    ->", col_price)
print("  rating   ->", col_rating)

# # If needed, override here, e.g.:
# col_category = "Category"
# col_price    = "Price"
# col_rating   = "Rating"

if col_category is None or col_price is None:
    raise ValueError("Please set col_category and col_price to valid column names and re-run.")



## 3) Clean price & rating
Converts price to numeric and rating to 0–5 range where possible.


In [ ]:

import re

def to_numeric_price(s):
    if pd.isna(s): return np.nan
    if isinstance(s, (int, float)): return float(s)
    s = str(s).replace(",", ".")
    s = re.sub(r"[^0-9.]", "", s)
    try:
        return float(s) if s else np.nan
    except:
        return np.nan

df_clean = df.copy()
df_clean["_price"] = df_clean[col_price].apply(to_numeric_price)
df_clean.loc[df_clean["_price"] < 0, "_price"] = np.nan

if col_rating in df_clean.columns:
    df_clean["_rating"] = pd.to_numeric(df_clean[col_rating], errors="coerce")
    df_clean.loc[(df_clean["_rating"] < 0) | (df_clean["_rating"] > 5), "_rating"] = np.nan
else:
    df_clean["_rating"] = np.nan

df_clean[[col_category, "_price", "_rating"]].head(5)



# Part 1 — Categories


In [ ]:

freq_cat = (df_clean[col_category]
            .astype(str).str.strip()
            .replace({"": np.nan}).dropna()
            .value_counts()
            .rename_axis("category")
            .reset_index(name="count"))

display(freq_cat.head(10))
freq_cat.to_csv("frequency_table_categories.csv", index=False)
print("Saved: frequency_table_categories.csv")


In [ ]:

import matplotlib.pyplot as plt

top_n = 15
subset = freq_cat.head(top_n)

plt.figure()
plt.bar(subset["category"], subset["count"])
plt.xticks(rotation=45, ha="right")
plt.title("Top categories by count")
plt.xlabel("Category")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
plt.pie(subset["count"], labels=subset["category"], autopct="%1.1f%%")
plt.title("Proportion of top categories")
plt.tight_layout()
plt.show()



# Part 2 — Prices


In [ ]:

price_series = df_clean["_price"].dropna()
print("Mean:", round(price_series.mean(), 2))
print("Median:", round(price_series.median(), 2))
print("Mode(s):", ", ".join(map(lambda x: str(round(x,2)), price_series.mode().head(5))))


In [ ]:

variance = price_series.var()
std_dev  = price_series.std()
min_p, max_p = price_series.min(), price_series.max()
price_range = max_p - min_p
q1, q3 = price_series.quantile([0.25, 0.75])
iqr = q3 - q1
print(f"Variance: {variance:.2f}\nStd Dev: {std_dev:.2f}\nRange: {price_range:.2f} (min={min_p:.2f}, max={max_p:.2f})\nIQR: {iqr:.2f}")


In [ ]:

plt.figure()
plt.hist(price_series, bins=50)
plt.title("Price distribution (all)")
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

p99 = price_series.quantile(0.99)
price_clip = price_series[price_series <= p99]

plt.figure()
plt.hist(price_clip, bins=50)
plt.title("Price distribution (<= 99th percentile)")
plt.xlabel("Price")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

plt.figure()
plt.hist(price_series[price_series > 0], bins=50)
plt.xscale("log")
plt.title("Price distribution (log x-axis)")
plt.xlabel("Price (log)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


In [ ]:

plt.figure()
plt.boxplot(price_series, vert=True, showfliers=True)
plt.title("Price — box plot (with outliers)")
plt.ylabel("Price")
plt.tight_layout()
plt.show()



# Part 3 — Ratings


In [ ]:

if df_clean["_rating"].notna().sum() == 0:
    print("No rating data — skipping Part 3.")
else:
    rating_series = df_clean["_rating"].dropna()
    print("Mean:", round(rating_series.mean(), 2))
    print("Median:", round(rating_series.median(), 2))
    print("Mode(s):", ", ".join(map(lambda x: str(round(x,2)), rating_series.mode().head(5))))


In [ ]:

if df_clean["_rating"].notna().sum() == 0:
    pass
else:
    r_var = rating_series.var()
    r_std = rating_series.std()
    r_q1, r_q3 = rating_series.quantile([0.25, 0.75])
    r_iqr = r_q3 - r_q1
    print(f"Variance: {r_var:.3f}  Std Dev: {r_std:.3f}  IQR: {r_iqr:.3f}")


In [ ]:

if df_clean["_rating"].notna().sum() == 0:
    pass
else:
    skew = rating_series.skew()
    kurt = rating_series.kurt()  # excess kurtosis
    print(f"Skewness: {skew:.3f} | Excess kurtosis: {kurt:.3f}")


In [ ]:

if df_clean["_rating"].notna().sum() == 0:
    pass
else:
    import matplotlib.pyplot as plt
    plt.figure()
    plt.hist(rating_series, bins=10)
    plt.title("Ratings distribution")
    plt.xlabel("Rating")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()



## 4) Business summary


In [ ]:

lines = []

if 'freq_cat' in locals() and not freq_cat.empty:
    total_products = int(freq_cat['count'].sum())
    top5_list = ", ".join(f"{row.category} ({row.count})" for _, row in freq_cat.head(5).iterrows())
    lines.append(f"Top categories (first 5): {top5_list}. Total listings: {total_products}.")
else:
    lines.append("Category summary unavailable.")

if 'price_series' in locals() and len(price_series) > 0:
    q1, q3 = price_series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lines.append(f"Average price ~£{price_series.mean():.2f} (median £{price_series.median():.2f}), range £{price_series.min():.2f}–£{price_series.max():.2f}, IQR ~£{iqr:.2f}.")
else:
    lines.append("Price analysis unavailable.")

if df_clean["_rating"].notna().sum() > 0:
    lines.append(f"Average rating {rating_series.mean():.2f} (median {rating_series.median():.2f}); skew={rating_series.skew():.2f}, kurtosis={rating_series.kurt():.2f}.")
else:
    lines.append("Rating analysis unavailable.")

report = "\n- ".join(["Business Summary:"] + lines)
print(report)
with open("business_summary.txt", "w", encoding="utf-8") as f:
    f.write(report)
print("\nSaved: business_summary.txt")
